In [1]:
import argparse

# parser is COMMENTED out because it doesn't work in jupyter notebook!!

########## START UNCOMMENT #################

# parser = argparse.ArgumentParser(description='Process some integers.')
# parser.add_argument("--condor", help="increase output verbosity")
# args = parser.parse_args()

# parser.add_argument('-ID',type=int,
#                    help='the ID of the model and the logs of the dexpression run)')

# args = parser.parse_args()
# print(args.ID)


import sys
# if args.condor:
#     print "Condor turned on"
#     sys.path.insert(0, '/esat/tiger/joramas/mscStudentsData/emotionModeling/libs/python2.7/site-packages')
#     sys.path.insert(1,'/esat/tiger/joramas/mscStudentsData/emotionModeling/libs/cuda_9.0/var/cuda-repo-9-0-local/usr/local/cuda-9.0/lib64')
#     sys.path.insert(2,'/esat/tiger/joramas/mscStudentsData/emotionModeling/libs/cudnn7/cudnn-9.0-linux-x64-v7/cuda/lib64')

#     print(sys.path)

# Give a run ID here. Change it to flags (arguments) in version 2.
ID = '4_1'

# if args.ID == None:
#     ID = '4_1'
# else:
#     ID = repr(args.ID)
#     print(args.ID)
    
RUNID = 'DeXpression_run_' + ID

########## STOP UNCOMMENT #################

import numpy as np
import tflearn
import tflearn.activations as activations
# Data loading and preprocessing
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.conv import avg_pool_2d, conv_2d, max_pool_2d
from tflearn.layers.core import dropout, flatten, fully_connected, input_data
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.normalization import batch_normalization


#chris library imports
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split

from test_recursive_image_load_V2 import load_CKP_data
from test_recursive_image_load_V2 import load_formated_data

from showNumpyInfo import showInfo


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:

# global Paths to define for each specific computer
#tf_checkpoints = where the checkpoints of tensorflow training algorithms are stored to be recovered if necessary
tf_checkpoints = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/tf_checkpoints"

# cascPath = the path to the cascade file for the facerecognition (relative paths didn't work on my windows edition)
cascPath = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/haarcascade.xml"



# Give a dropout if required (change to True and define the dropout percentage).
dropout = False
dropout_keep_prob=0.5

# Load data from: https://drive.google.com/drive/folders/1YWT8DJivNOZzQRPCiHDPY0LL_dymdQIS?usp=sharing
X_data = np.load('../data/CKP_X.npy')
Y_data = np.load('../data/CKP_Y.npy')
X_subID = (np.load('../data/CKP_subjectIds.npy')).astype('uint8')

X_reduced = X_data.astype('uint8')
Y_reduced = Y_data

# showInfo(X_reduced,"X_reduced")
# showInfo(Y_reduced,"Y_reduced")

# showInfo(X_subID,"X_subID")

In [3]:
# seperate the dataset in 11 parts that do not share users: 
#   9 parts train set
#   1 part validation set
#   1 part test set
# allows future possibility for 10-fold crossover

partSize = (np.floor(len((X_subID))/12)).astype('uint8')
print("partsize " , partSize)

lastSubID = 0 #the ID of the subject of the last instance
partCount = 0 #The amount of instances in this part already
partID = 0    #keeps the number of the part that is being filled right now

X_array = np.asarray([])
Y_array = np.asarray([])
sub_array = np.asarray([])

X_parts = []
Y_parts = []
X_subID_parts = []  

#iterate through al the ID's if a part is full and the subjectID is different from the previous, fill the next part 
for i in range(0,len(X_subID)):

#     print("this part IDs top border ", partSize ," and partCount = ", partCount)
    if((partCount >= partSize) & (lastSubID != X_subID[i])):
      
        
        #fill the parts to the partslists
        X_parts.append(X_array)
        Y_parts.append(Y_array)
        X_subID_parts.append(sub_array)
        
        #refresh the temporary arrays
        X_array = np.asarray([])
        Y_array = np.asarray([])
        sub_array = np.asarray([])
        #reset counter & set next part to be filled
        partCount = 0
        partID = partID + 1
        
        
    X_array = np.append(X_array ,X_reduced[i])
    Y_array = np.append(Y_array ,Y_reduced[i])
    sub_array = np.append(sub_array ,X_subID[i])  
    

    partCount = partCount +1
    lastSubID = X_subID[i]

print(len(X_parts))
print(len(Y_parts))
print(len(X_subID_parts))

partsize  27
11
11
11


In [4]:
# Just test output

# print(len(X_subID))
# print("partsize", partSize)

# for o in range(0,len(X_parts)):
#     print (o)
#     showInfo(X_parts[o].reshape(-1,224,224,1))
#     showInfo(Y_parts[o].reshape(-1,7))
#     showInfo(X_subID_parts[o])

In [5]:
#select 9 from the 11 parts (part 11 is usually the smallest so it is bad as test or validation data)
select =  [0,1,2,3,6,7,8,9,10]
select_val = 4
select_test= 5

if((len(select)+1 + 1)!= len(X_parts)):
    print("Warning it is possible that not all parts of the dataset are used")

X = np.asarray([])
Y = np.asarray([])
subID = np.asarray([])

# create the trainings set X and Y (SubID is not given to the network)
for i in select:
    X = np.append(X,X_parts[i])
    Y = np.append(Y,Y_parts[i])
    subID = np.append(subID,X_subID_parts[i])

X = (X.reshape(-1,224,224,1)).astype('uint8')
Y = (Y.reshape(-1,7)).astype('uint8')
subID = subID.astype('uint8')

# create the validation set X_val and Y-val (SubID_val is not given to the network)
X_val = X_parts[select_val].reshape(-1,224,224,1).astype('uint8')
Y_val = Y_parts[select_val].reshape(-1,7).astype('uint8')
subID_val = X_subID_parts[select_val].astype('uint8')

# create the test set X_test and Y_test (SubID_test is not given to the network)
X_test = X_parts[select_test].reshape(-1,224,224,1).astype('uint8')
Y_test = Y_parts[select_test].reshape(-1,7).astype('uint8')
subID_test = X_subID_parts[select_test].astype('uint8')

# save the array with the Id's of each user per set, for future reference
np.save('train_subject_ID.npy',subID)
np.save('validation_subject_ID.npy',subID_val)
np.save('test_subject_ID.npy',subID_test)


# just test outpt

# showInfo(X,"X")
# showInfo(Y,"Y")
# showInfo(X_val,"X_vall")
# showInfo(Y_val,"Y_vall")
# showInfo(X_test,"X_test")
# showInfo(Y_test,"Y_test")

# for i in range(0,len(X)):
#     str = "index " + repr(i) + " X  "+ repr(subID[i]) + " emotion " +  repr(Y[i])
#     print(str)
#     cv2.imshow(str, X[i].reshape((224,224)))
#     cv2.waitKey(0)
# cv2.destroyAllWindows()


In [6]:
showInfo(X , "X")

# cv2.imshow("example", X[1].reshape((224,224)))
# cv2.waitKey(0)

print("size labels shape: " + repr(Y.shape))
print("type labels: " + repr(type(Y)))


Name  X
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (249, 224, 224, 1)
size labels shape: (249, 7)
type labels: <class 'numpy.ndarray'>


In [7]:
# Define number of output classes.
num_classes = 7

# Define padding scheme.
padding = 'VALID'

# Model Architecture
network = input_data(shape=[None, 224, 224, 1])
conv_1 = relu(conv_2d(network, 64, 7, strides=2, bias=True, padding=padding, activation=None, name='Conv2d_1'))
maxpool_1 = batch_normalization(max_pool_2d(conv_1, 3, strides=2, padding=padding, name='MaxPool_1'))
LRN_1 = local_response_normalization(maxpool_1, name='LRN_1')
# FeatEX-1
conv_2a = relu(conv_2d(maxpool_1, 96, 1, strides=1, padding=padding, name='Conv_2a_FX1'))
maxpool_2a = max_pool_2d(maxpool_1, 3, strides=1, padding=padding, name='MaxPool_2a_FX1')
conv_2b = relu(conv_2d(conv_2a, 208, 3, strides=1, padding=padding, name='Conv_2b_FX1'))
conv_2c = relu(conv_2d(maxpool_2a, 64, 1, strides=1, padding=padding, name='Conv_2c_FX1'))
FX1_out = merge([conv_2b, conv_2c], mode='concat', axis=3, name='FX1_out')
# FeatEX-2
conv_3a = relu(conv_2d(FX1_out, 96, 1, strides=1, padding=padding, name='Conv_3a_FX2'))
maxpool_3a = max_pool_2d(FX1_out, 3, strides=1, padding=padding, name='MaxPool_3a_FX2')
conv_3b = relu(conv_2d(conv_3a, 208, 3, strides=1, padding=padding, name='Conv_3b_FX2'))
conv_3c = relu(conv_2d(maxpool_3a, 64, 1, strides=1, padding=padding, name='Conv_3c_FX2'))
FX2_out = merge([conv_3b, conv_3c], mode='concat', axis=3, name='FX2_out')
net = flatten(FX2_out)
if dropout:
    net = dropout(net, dropout_keep_prob)
loss = fully_connected(net, num_classes,activation='softmax')

# Compile the model and define the hyperparameters
network = tflearn.regression(loss, optimizer='Adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0001)

#create a custom tensorflow session to manage the used resources
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)


# Final definition of model checkpoints and other configurations
#model = tflearn.DNN(network, checkpoint_path='/home/cc/DeXpression/DeXpression_checkpoints',
model = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
                    max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
tf.RunOptions(report_tensor_allocations_upon_oom = True)

report_tensor_allocations_upon_oom: true

In [9]:
# Fit the model, train for 20 epochs. (Change all parameters to flags (arguments) on version 2.)
#model.fit(X, Y, n_epoch=20, validation_set=0.1, shuffle=True, show_metric=True, batch_size=50, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)
model.fit(X, Y, n_epoch=20, validation_set=(X_test,Y_test), shuffle=True, show_metric=True, batch_size=50, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)

# Save the model
model.save(tf_checkpoints + '/' + RUNID + '.model')
print("finished training and saving")

# Load the model if required, later.
#model.load('./DeXpression_checkpoints/' + RUNID + '.model')

Training Step: 4  | total loss: 9.84865 | time: 46.269s
| Adam | epoch: 001 | loss: 9.84865 - acc: 0.2757 -- iter: 200/249
Training Step: 5  | total loss: 11.90586 | time: 79.027s
| Adam | epoch: 001 | loss: 11.90586 - acc: 0.2095 | val_loss: 7.26553 - val_acc: 0.3704 -- iter: 249/249
--


KeyboardInterrupt: 